In [1]:
from pyspark import *
from pyspark.sql import types

In [2]:
sc = SparkContext(master="local", appName="test")
sc

<SparkContext master=local appName=test>

In [3]:
import urllib


In [22]:
data = urllib.request.urlretrieve('http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz', 'data_10_percent.gz')

URLError: <urlopen error [Errno -3] Temporary failure in name resolution>

In [6]:
data_file = './data_10_percent.gz'

In [7]:
lines = sc.textFile(data_file)

In [8]:
from pyspark.sql import SQLContext

In [9]:
sqlContext = SQLContext(sc)
sqlContext

In [10]:
lines.take(3)

['0,tcp,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,9,9,1.00,0.00,0.11,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,19,19,1.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,235,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,29,29,1.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,normal.']

In [11]:
parts = lines.map(lambda l: l.split(","))

In [12]:
parts.take(3)

[['0',
  'tcp',
  'http',
  'SF',
  '181',
  '5450',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '8',
  '8',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '1.00',
  '0.00',
  '0.00',
  '9',
  '9',
  '1.00',
  '0.00',
  '0.11',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  'normal.'],
 ['0',
  'tcp',
  'http',
  'SF',
  '239',
  '486',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '8',
  '8',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '1.00',
  '0.00',
  '0.00',
  '19',
  '19',
  '1.00',
  '0.00',
  '0.05',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  'normal.'],
 ['0',
  'tcp',
  'http',
  'SF',
  '235',
  '1337',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '8',
  '8',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '1.00',
  '0.00',
  '0.00',
  '29',
  '29',
  '1.00',
  '0.00',


In [13]:
schema = types.StructType([
        types.StructField('duration', types.IntegerType()),
        types.StructField('service', types.StringType()),
        types.StructField('protocol_type', types.StringType()),
        types.StructField('flag', types.StringType()),
        types.StructField('src_bytes', types.IntegerType()),
        types.StructField('dst_bytes', types.StringType()),
        types.StructField('interactions', types.StringType()),
    ])
schema

StructType(List(StructField(duration,IntegerType,true),StructField(service,StringType,true),StructField(protocol_type,StringType,true),StructField(flag,StringType,true),StructField(src_bytes,IntegerType,true),StructField(dst_bytes,StringType,true),StructField(interactions,StringType,true)))

In [14]:
objet = parts.map(lambda key:(int(key[0]),key[1],key[2],key[3],int(key[4]),int(key[5]),key[-1]))

In [15]:
schema2 = sqlContext.createDataFrame(objet, schema)
schema2.show(10)

+--------+-------+-------------+----+---------+---------+------------+
|duration|service|protocol_type|flag|src_bytes|dst_bytes|interactions|
+--------+-------+-------------+----+---------+---------+------------+
|       0|    tcp|         http|  SF|      181|     5450|     normal.|
|       0|    tcp|         http|  SF|      239|      486|     normal.|
|       0|    tcp|         http|  SF|      235|     1337|     normal.|
|       0|    tcp|         http|  SF|      219|     1337|     normal.|
|       0|    tcp|         http|  SF|      217|     2032|     normal.|
|       0|    tcp|         http|  SF|      217|     2032|     normal.|
|       0|    tcp|         http|  SF|      212|     1940|     normal.|
|       0|    tcp|         http|  SF|      159|     4087|     normal.|
|       0|    tcp|         http|  SF|      210|      151|     normal.|
|       0|    tcp|         http|  SF|      212|      786|     normal.|
+--------+-------+-------------+----+---------+---------+------------+
only s

In [16]:
view = schema2.createOrReplaceTempView("Interaction")
#Une vue sert à copier les données d'un schema sans la modifier 

In [17]:
schema2.columns

['duration',
 'service',
 'protocol_type',
 'flag',
 'src_bytes',
 'dst_bytes',
 'interactions']

In [21]:
res = schema2[ (schema2['protocol_type'] == 'tcp') & (schema2['duration']>1000) & (schema2['dst_bytes'] == 0) ].select(schema2['duration'], schema2['dst_bytes'])
res.show()

+--------+---------+
|duration|dst_bytes|
+--------+---------+
+--------+---------+



In [20]:
res.count()

0